训练模型是个迭代过程，在每个迭代（也称之为 `epoch`）中，该模型对输出进行猜测，计算其猜测中的误差（损失），收集误差相对于其参数的导数（如我们在前面的模块中看到的），并使用梯度下降优化这些参数。

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

NameError: name '_C' is not defined

In [ ]:
training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

`超参` 指的是那些在训练过程中明确设置的参数。这些参数用于控制训练过程本身的行为，而不是模型内部的参数值。我们在本文中设置的超参数如下：
- `Epoch` 的数量——训练数据集上迭代的次数。
- `Batch size`——每次迭代中传递给模型的数据样本数量。
- `Learning rate`——优化器内部使用的更新模型参数的大小。较大的值可能会导致收敛过程中的振荡，较小的值可能会导致收敛速度过慢。

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

损失函数的作用：
- 损失函数衡量得到的结果与目标值的不同程度
- 损失函数的梯度有助于优化器在训练期间进行适当的参数调整
- 损失函数是在训练过程中以最小化为目标

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在迭代训练中，通常由三步构成：
- 调用 `optimizer.zero_grad()` 重置模型参数的梯度。默认情况下，每次调用 `.backward()` ，梯度都会累积（即不会被重写）。
- 调用 `loss.backward()` 计算当前迭代的损失相对于模型参数的梯度。
- 调用 `optimizer.step()` 更新内部参数。

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "./data/model.pth")

print("Saved PyTorch Model State to model.pth")